In [ ]:
# Install dependencies if needed
!pip install langgraph bedrock-agentcore

In [1]:
import os
import uuid
import json
from typing import Dict, Any, List, Optional

from langgraph.prebuilt import create_react_agent

from langchain_aws.memory import (
    create_store_messages_tool,
    create_search_memory_tool,
    create_list_messages_tool
)
from bedrock_agentcore.memory import MemoryClient

In [2]:
REGION = "us-east-1" 

In [3]:
memory_client = MemoryClient(region_name=REGION)

In [10]:
create_response = memory_client.create_memory_and_wait(
    name="CustomerSupportAgentMemory",
    strategies=[{
        "summaryMemoryStrategy": {
            "name": "SessionSummarizer",
            "namespaces": ["/summaries/{actorId}/{sessionId}"]
        }
    }]
)

In [4]:
MEMORY_ID = create_response["id"]
ACTOR_ID = "User99"
SESSION_ID = "OrderSupportSession5"

In [5]:
def list_events():
    events = memory_client.list_events(memory_id=MEMORY_ID, session_id=SESSION_ID, actor_id=ACTOR_ID)
    for event in events:
        print(f"\n================================ Event Id: {event['eventId']} =================================\n")
        for payload in event['payload']:
            print(f"================================ {payload['conversational']['role']} =================================")
            print(payload['conversational']['content']['text'])

In [6]:
list_events()

In [7]:
store_messages_tool = create_store_messages_tool(
    memory_client=memory_client
)

search_memory_tool = create_search_memory_tool(
    memory_client=memory_client
)

list_messages_tool = create_list_messages_tool(
    memory_client=memory_client
)

In [8]:
from langchain_core.tools import tool

@tool
def lookup_order(order_id: str) -> str:
    """Returns the order status"""
    return f"Customer order {order_id} was shipped 3 days ago, with an expected delivery tomorrow before 5pm"

@tool
def update_customer_email(email: str):
    """Updates customer's email address"""
    return f"Customer's email updated to {email}"

In [9]:
llm_model = "bedrock_converse:us.anthropic.claude-3-5-haiku-20241022-v1:0"
prompt = """You are are customer service assistant with access to order and memory tools."""

agent = create_react_agent(
    llm_model,
    tools=[store_messages_tool, search_memory_tool, list_messages_tool, lookup_order, update_customer_email],
    prompt=prompt
)

In [10]:
config = {
    "configurable": {
        "memory_id": MEMORY_ID,
        "session_id": SESSION_ID,
        "actor_id": ACTOR_ID,
        "namespace": f"/summaries/{ACTOR_ID}/{SESSION_ID}"
    }
}

In [11]:
response = agent.invoke({
    "messages": [
        "Hi, I'm having trouble with my order #12345"
    ]
}, config)

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm having trouble with my order #12345
================================== Ai Message ==================================

[{'type': 'text', 'text': "I'll help you look up the status of your order right away."}, {'type': 'tool_use', 'name': 'lookup_order', 'input': {'order_id': '12345'}, 'id': 'tooluse_ub-cWkE_Tc-2ItPbhbqT0A'}]
Tool Calls:
  lookup_order (tooluse_ub-cWkE_Tc-2ItPbhbqT0A)
 Call ID: tooluse_ub-cWkE_Tc-2ItPbhbqT0A
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: lookup_order

Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================== Ai Message ==================================

[{'type': 'text', 'text': "I can see that your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific about the order th

In [12]:
list_events()


================================ Event Id: 0000001753929112000#5996cf1c =================================

================================ USER =================================
Hi, I'm having trouble with my order #12345
================================ ASSISTANT =================================
I'll help you look up the status of your order right away. Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm. I can see that your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific about the order that is causing you concern? I'm here to help you with any questions or issues you might be experiencing.


In [13]:
response = agent.invoke({
    "messages": [
        "Actually, before that - I also want to change my email address"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Actually, before that - I also want to change my email address
================================== Ai Message ==================================

I can help you update your email address. Could you please provide me with the new email address you would like to use?

I'll use the `update_customer_email` tool to make this change for you once you share the new email address.


In [14]:
list_events()


================================ Event Id: 0000001753929112000#5996cf1c =================================

================================ USER =================================
Hi, I'm having trouble with my order #12345
================================ ASSISTANT =================================
I'll help you look up the status of your order right away. Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm. I can see that your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific about the order that is causing you concern? I'm here to help you with any questions or issues you might be experiencing.


In [15]:
response = agent.invoke({
    "messages": [
        "myemail@example.com"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

myemail@example.com
================================== Ai Message ==================================

[{'type': 'text', 'text': "I'll help you update your email address. I'll use the `update_customer_email` tool to do this."}, {'type': 'tool_use', 'name': 'update_customer_email', 'input': {'email': 'myemail@example.com'}, 'id': 'tooluse_m-5o4fUXRrG4zamhpoEW2g'}]
Tool Calls:
  update_customer_email (tooluse_m-5o4fUXRrG4zamhpoEW2g)
 Call ID: tooluse_m-5o4fUXRrG4zamhpoEW2g
  Args:
    email: myemail@example.com
================================= Tool Message =================================
Name: update_customer_email

Customer's email updated to myemail@example.com
================================== Ai Message ==================================

Your email address has been successfully updated to myemail@example.com. Is there anything else I can help you with today?


In [16]:
list_events()


================================ Event Id: 0000001753929112000#5996cf1c =================================

================================ USER =================================
Hi, I'm having trouble with my order #12345
================================ ASSISTANT =================================
I'll help you look up the status of your order right away. Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm. I can see that your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific about the order that is causing you concern? I'm here to help you with any questions or issues you might be experiencing.


In [17]:
response = agent.invoke({
    "messages": [
        "The package already arrived, but seems damaged"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

The package already arrived, but seems damaged
================================== Ai Message ==================================

I'm sorry to hear that your package arrived damaged. I'll help you resolve this issue. Could you please provide me with your order number so I can look up the details of your order and assist you further?


In [18]:
list_events()


================================ Event Id: 0000001753929112000#5996cf1c =================================

================================ USER =================================
Hi, I'm having trouble with my order #12345
================================ ASSISTANT =================================
I'll help you look up the status of your order right away. Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm. I can see that your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific about the order that is causing you concern? I'm here to help you with any questions or issues you might be experiencing.
